CREATE FUNCTION FOR CITY LATITUDE AND LONGITUDE

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates

def get_city_data(cities):

    city_data = []

    for city in cities:
        url= f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

        try:
            country = city_soup.find(class_='infobox-data').get_text()
            city_longitude = city_soup.find(class_='longitude').get_text()
            city_latitude = city_soup.find(class_='latitude').get_text()

        except AttributeError:
            country, latitude, longitude = None, None, None  # Handle missing values

        # extract data
        city_data.append({
            "city_name" : city,
            "Country" : country,
            "Latitude" : parse(city_latitude),
            "Longitude" : parse(city_longitude)
        })

    return pd.DataFrame(city_data)

CALL FUNCTION

In [2]:
cities = ["Berlin", "Hamburg", "Munich"]
city_df = get_city_data(cities)
city_df

,city_name,Country,Latitude,Longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


CREATE FUNCTION FOR POPULATION DATA

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime # to get today's date

def Get_Population_Data(cities):

    population_data = []
    # make a loop
    for city in cities:
        url = f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        Pop_soup = BeautifulSoup(response.content, 'html.parser')

        # extract the data
        try:
            city_pop = Pop_soup.find(string='Population').find_next('td').get_text(strip=True)
            city_pop_clean = int(city_pop.replace(",", ""))
            today = datetime.today().strftime("%d.%m.%Y")

        except AttributeError:
            city_pop_clean = None, None, None  # Handle missing values

        # values for each city
        population_data.append({
            "city_name" : city,
            "Population": city_pop_clean,
            "Population_Timestamp":today
        })

    return pd.DataFrame(population_data)

CALL FUNCTION

In [4]:
cities = ["Berlin", "Hamburg", "Munich"]
population = Get_Population_Data(cities)
population

,city_name,Population,Population_Timestamp
0,Berlin,3596999,23.05.2025
1,Hamburg,1964021,23.05.2025
2,Munich,1510378,23.05.2025


ESTABLISH CONNECTION WITH SQL

In [4]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment

schema = "city_workshop"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD")
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

SEND CITY DATA TO SQL DATA BASE; FIRST CREATE TABLE AND ENSURE ALL THE TABLE NAME ARE IN THE SAME FORMAT

city_df.to_sql('city',
                  if_exists='append',
                  con=connection_string,
                  index=False)

RETRIEVE DATA FROM SQL

In [6]:
city_df_from_sql = pd.read_sql("city", con=connection_string)
city_df_from_sql

,City_id,city_name,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


PREPARING TO SEND POPULATION DATA TO SQL
FIRST MERGE CITY AND POPULATION

In [7]:
population_df_raw = population.merge(city_df_from_sql, on ="city_name", how= "left")
population_df_raw 

,city_name,Population,Population_Timestamp,City_id,Country,Latitude,Longitude
0,Berlin,3596999,23.05.2025,1,Germany,52.5200,13.405
1,Hamburg,1964021,23.05.2025,2,Germany,53.5500,10.000
2,Munich,1510378,23.05.2025,3,Germany,48.1375,11.575


SELECT APPROPRAITE COLUMN FOR YOUR POPULATION TABLE

In [8]:
population_df =population_df_raw[["Population", "Population_Timestamp", "City_id"]]
population_df

,Population,Population_Timestamp,City_id
0,3596999,23.05.2025,1
1,1964021,23.05.2025,2
2,1510378,23.05.2025,3


SEND POPULATION DATA TO SQL DATA BASE; FIRST CREATE TABLE AND ENSURE ALL THE TABLE NAME ARE IN THE SAME FORMAT

In [ ]:
#population_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

RETRIEVE POPULATION DATA

In [5]:
population_df_from_sql = pd.read_sql("population", con=connection_string)
population_df_from_sql

,Population,Population_Timestamp,city_id
0,3596999,21.05.2025,1
1,1964021,21.05.2025,2
2,1510378,21.05.2025,3
